# Preprocessing des données

corpus de test : 

In [1]:
import pandas as pd
corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)
corpus_sephora3k.head()
text = corpus_sephora3k[corpus_sephora3k['language'] == 'en']['text'].tolist()

C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_22620\735203815.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  corpus_sephora3k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/corpus_sephora3k.csv",sep=";", error_bad_lines=False)


#### Initialisation pipe spaCy en anglais

`python -m spacy download en_core_web_sm`

In [26]:
import spacy

nlp = spacy.load("en_core_web_sm")

#### Extraction des emojis

Ajout de **emoji** dans la pipe spaCy

In [27]:
from spacymoji import Emoji


nlp.add_pipe("emoji", first=True)

In [8]:
#clarascanner/blob/main/notebooks/cleaning.ipynb


#------------------------ /!\ Ne fonctionne pas------------------------------
from emoji import UNICODE_EMOJI 

def emojis_extraction(text):
    emojis = []
    for x in text:
        if x in UNICODE_EMOJI['en']:
            emojis.append(x)
            text = text.replace(x, f' {x} ')
    return (text,emojis)

#-----------------------------------------------------------------------------
import re
def clean_emojis(txt): #--chaine de caractères
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U0001F1F2-\U0001F1F4"  # Macau flag
            u"\U0001F1E6-\U0001F1FF"  # flags
            u"\U0001F600-\U0001F64F"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U0001F1F2"
            u"\U0001F1F4"
            u"\U0001F620"
            u"\u200d"
            u"\u2640-\u2642"
            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',txt)

#### Identification des doublons
Afin d'éviter d'analyser le texte de post identiques, nous allons ici supprimer les doublons présent dans notre corpus de texte.

Pour ce faire nous allon utiliser la librairie textdistance (`pip install text-distance`), et nottament la distance normalisée `textdistance.hamming.normalized_distance` qui nous permettra de définir un seuil entre 0 et 1.
Nous pouvons également utiliser la librairie `fuzzywuzzy`, qui va nous permettre grâce à la fonction *token_sort_ratio* de mesurer la similitude entre deux chaines de caractères malgré la position des mots dans la chaines.

In [10]:
import textdistance
import numpy as np

On doit enlever les hashtags temporairement afin de vérifier s'il existe des doublons, car en effet des hashtag différent sur deux posts qui sont en fait identiques peuvent perturber le tri.

Fonction qui retourne notre texte sans les # : 

In [11]:
def clean_hashtag(t): #--text
    hashtag_pattern= re.compile("#[A-Za-z0-9_]+")
    return re.sub(hashtag_pattern,"", t) #On supprime tout les types de #

Une fois les hashtags retiré, il nous faut calculer la distance entre tout les éléments du corpus. La fonction suivante va calculer la distance entre les élements de la liste (corpus de texte) et ensuite supprimer les élements qui sont présent plusieurs fois :

In [17]:
# /!\ Renvoie une matrice de distance, pas opti
#------------------------------------------------------------------------------------------
def dist_mat(t): #-- text
    dist_matrix = np.zeros((len(t),len(t)),dtype=np.float)
    for i in range(len(t)):
        for j in range(len(t)):
            dist_matrix[i,j] = hamming(t[i],t[j])
    return dist_matrix
#------------------------------------------------------------------------------------------




def del_double(txt,s,method): # --text --seuil (s dans [0;1] ou [0;100] selon la méthode) --méthode (hamming-levenshtein.normalized_distance ou fuzzywuzzy-token_sort_ratio)
    t = txt.copy()
    if(method == "hamming-levenshtein.normalized_distance"): #Ici deux texte identique dist = 0
        hamming = textdistance.hamming.normalized_distance #initialisiation de hamming avec la distance normalisée.
        i = 0
        r = len(t)
        while(i<r):
            r = len(t)
            j=i+1
            while(j<r):
                if(hamming(clean_emojis(clean_hashtag(t[i])).strip(),clean_emojis(clean_hashtag(t[j])).strip() <= s ): # Si la distance entre les deux élemens de la liste inf à seuil
                    del t[j] #delete
                    r = len(t) #on actualise la taille de la listes
                else:
                    j+=1
            i+=1
    if(method == "fuzzywuzzy-token_sort_ratio"): #Ici deux texte identique ratio = 100
        i = 0
        r = len(t)
        while(i<r):
            r = len(t)
            j=i+1
            while(j<r):
                if(token_sort_ratio(clean_emojis(clean_hashtag(t[i])),clean_emojis(clean_hashtag(t[j]))) >= s ): # Si la distance entre les deux élemens de la liste inf à seuil
                    del t[j] #delete
                    r = len(t) #on actualise la taille de la listes
                else:
                    j+=1
            i+=1
    return t

# Supprimer les doublons en fonction du timestamp


#### Ratio mot/hashtag
Calcul du ratio mot/hastag dans un post

In [6]:
import spacy 
from spacy.lang.en import English

(OPTIONEL) Ici nous ajoutons le token 'hashtag' dans la pipe du modèle spacy.

In [29]:

#  /!\ Ne marche pas
#--------------------------------------------------------------------------------------------
from spacy.vocab import Vocab
from spacy.language import Language

@Language.component("hashtag")
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token_index,token in enumerate(doc):
            if token.text == '#':
                if token.head is not None:
                    start_index = token.i
                    end_index = start_index + 1
                    print(start_index, end_index)
                    with doc.retokenize() as retokenizer:
                        if retokenizer.merge(doc[start_index:end_index]) is not None: #PB avec retokenize
                            merged_hashtag = True
                            break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

#-----------------------------------------------------------------------------------------------------------

In [31]:
nlp.add_pipe("hashtag")

<function __main__.hashtag_pipe(doc)>

In [32]:
nlp('Hello #Hello')

6 12


ValueError: [E199] Unable to merge 0-length span at `doc[3:3]`.

Afin de calculer le ration mot/hashtags, nous allons claculer le nombre de mot, cependant en utilisant la pipe spacy classique, un '#' est un token en lui-même. Nous devons donc calculer le nom de de '#' et le soustraire au nombre total de mot, car en effet tout les mot précédé d'un '#' seront considéré comme de "vrai' mots.

In [98]:
from spacy.lang.en import English


def del_junk_post(txt,s): # --text --seuil (s dans [0;1], le seuil 1 permet de supprimer un post composé à 100% de hashtag)
    t = txt.copy()
    nlp = English()
    i = 0
    r = len(t)
    while(i<r):
        r = len(t)
        nb_hash = 0
        nb_word = 0
        doc = nlp(t[i]) # On tokenize l'element de notre liste
        for token in doc: # Pour chaque token de la liste
            if(token.text == '#'):
                nb_hash+=1
            else:
                nb_word+=1
        if(nb_word-nb_hash==0): # On vérifie d'abord que le nombre de mot n'est pas nul, auquel cas nous supprimmons le post
            del t[i]
            r = len(t)
        elif(nb_hash/(nb_word-nb_hash) > s): # On calcul le ratio, si inférieur au seuil => on delete
            del t[i]
            r = len(t)
        i+=1
    return t


In [123]:
tst = ['a #salut #bonjour #hello', 'Bonjour à tous','je je je je #salut #bonjour #hello','Salut','#bjr #llala #kdkdd', 'tutu tata toto']
del_junk_post(tst,0.4)

['Bonjour à tous', 'Salut', 'tutu tata toto']

#### Traitement des hashtag restant

In [135]:
import wordninja

In [140]:
wordninja.split('helloilovesephora')

['hello', 'i', 'love', 'sep', 'hora']

In [ ]:
 #/!\ Custom le language en ajoutant les marques !

check les seuils , décomposer les mot dans les hashtag, position du hashtag dans le post, dictionnaire frequence langue anglaise